##**Import Librarys**

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt

##**Import  Excel files for Datasets**








In [2]:
order_report = pd.read_excel('./Company X - Order Report.xlsx')

In [3]:
df = order_report['ExternOrderNo']

In [4]:
order_report.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ExternOrderNo  400 non-null    int64  
 1   SKU            400 non-null    object 
 2   Order Qty      400 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 9.5+ KB


In [5]:
order_report.head()

,ExternOrderNo,SKU,Order Qty
0,2001827036,8904223818706,1.0
1,2001827036,8904223819093,1.0
2,2001827036,8904223819109,1.0
3,2001827036,8904223818430,1.0
4,2001827036,8904223819277,1.0


In [6]:
sku_master = pd.read_excel('./Company X - SKU Master.xlsx')
sku_master.head()

,SKU,Weight (g)
0,8904223815682,210
1,8904223815859,165
2,8904223815866,113
3,8904223815873,65
4,8904223816214,120


In [7]:
df_pincode_zone = pd.read_excel('./Company X - Pincode Zones.xlsx')

In [8]:
df_pincode_zone

,Warehouse Pincode,Customer Pincode,Zone
0,121003,507101,d
1,121003,486886,d
2,121003,532484,d
3,121003,143001,b
4,121003,515591,d
...,...,...,...
119,121003,325207,b
120,121003,303702,b
121,121003,313301,b
122,121003,173212,e


In [9]:
df_invoice = pd.read_excel('./Courier Company - Invoice.xlsx')

In [10]:
df_invoice.head()

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4


In [11]:
df_rates = pd.read_excel('./Courier Company - Rates.xlsx')

In [12]:
df_rates

,Zone,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge
0,A,0.50,29.5,23.6,13.6,23.6
1,B,1.00,33.0,28.3,20.5,28.3
2,C,1.25,40.1,38.9,31.9,38.9
3,D,1.50,45.4,44.8,41.3,44.8
4,E,2.00,56.6,55.5,50.7,55.5


#**Find totel weight in (kg) for each order number**

**Merging two datframes sku_master and order_report on the basis of SKU column after then applying  group by on the basis of order number**

In [13]:
temp_df  = sku_master.merge(order_report,how = 'inner',on = 'SKU')
temp_df['totl_weight (Kg)'] = (temp_df['Weight (g)']*temp_df['Order Qty'])/1000
#temp_df['totel_item_price'] = temp_df['Order Qty'] *temp_df['Item Price(Per Qty.)']
temp_df.drop_duplicates()

temp_df = temp_df.groupby('ExternOrderNo',as_index = False ).sum('totl_weight (Kg),totel_item_price')[['ExternOrderNo','totl_weight (Kg)']]

In [14]:
temp_df=temp_df.merge(df).drop_duplicates()

In [15]:
temp_df

,ExternOrderNo,totl_weight (Kg)
0,2001806210,0.220
2,2001806226,0.480
3,2001806229,0.500
6,2001806232,1.302
8,2001806233,0.245
...,...,...
380,2001821995,0.477
382,2001822466,1.376
384,2001823564,0.672
387,2001825261,1.557


#**Merging df_invoice with temp_df and concate with df_pincode_zone**

In [16]:
temp_invoice = df_invoice.merge(temp_df,how = 'inner',left_on = 'Order ID',right_on = 'ExternOrderNo')
final_df= pd.concat([temp_invoice,df_pincode_zone],axis = 1)

In [17]:
final_df

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),ExternOrderNo,totl_weight (Kg),Warehouse Pincode,Customer Pincode,Zone
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,2001806232,1.302,121003,507101,d
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2,2001806273,0.615,121003,486886,d
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,2001806408,2.265,121003,532484,d
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3,2001806458,0.700,121003,143001,b
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4,2001807012,0.240,121003,515591,d
...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,1091118551656,2001812941,0.73,121003,325207,d,Forward charges,90.2,2001812941,0.500,121003,325207,b
120,1091117614452,2001809383,0.50,121003,303702,d,Forward and RTO charges,86.7,2001809383,0.607,121003,303702,b
121,1091120922803,2001820978,0.50,121003,313301,d,Forward charges,45.4,2001820978,0.515,121003,313301,b
122,1091121844806,2001811475,0.50,121003,173212,b,Forward charges,33.0,2001811475,0.689,121003,173212,e


In [18]:
final_df.columns.values[12] = 'Zone_by_company'
final_df.head(10)

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),ExternOrderNo,totl_weight (Kg),Warehouse Pincode,Customer Pincode,Zone_by_company
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,2001806232,1.302,121003,507101,d
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2,2001806273,0.615,121003,486886,d
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,2001806408,2.265,121003,532484,d
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3,2001806458,0.700,121003,143001,b
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4,2001807012,0.240,121003,515591,d
5,1091117229555,2001806686,0.15,121003,326502,d,Forward charges,45.4,2001806686,0.240,121003,326502,d
6,1091117229776,2001806885,1.00,121003,208019,b,Forward charges,61.3,2001806885,0.840,121003,208019,b
7,1091117323112,2001807058,1.15,121003,140301,b,Forward charges,89.6,2001807058,1.168,121003,140301,b
8,1091117323812,2001807186,0.50,121003,396001,d,Forward charges,45.4,2001807186,0.500,121003,396001,d
9,1091117324206,2001807290,0.50,121003,711106,d,Forward charges,45.4,2001807290,0.500,121003,711106,d


#**Calculate the excpeted charges by the company X**

In [19]:
def slab_calculate(row):
    d = row
    if d['Zone'] == 'd':
        slab = np.ceil(d['Charged Weight'] / 1.25) * 1.25
    elif d['Zone'] == 'a':
        slab = np.ceil(d['Charged Weight'] / 0.25) * 0.25
    elif d['Zone'] == 'b':
        slab = np.ceil(d['Charged Weight'] / 0.50) * 0.50
    elif d['Zone'] == 'c':
        slab = np.ceil(d['Charged Weight'] / 0.75) * 0.75
    elif d['Zone'] == 'e':
        slab = np.ceil(d['Charged Weight'] / 1.00) * 1.00
    return slab

final_df['weight_slab'] = final_df.apply(lambda x: slab_calculate(x), axis=1)


In [20]:
final_df.head()

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),ExternOrderNo,totl_weight (Kg),Warehouse Pincode,Customer Pincode,Zone_by_company,weight_slab
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,2001806232,1.302,121003,507101,d,2.50
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2,2001806273,0.615,121003,486886,d,1.25
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,2001806408,2.265,121003,532484,d,2.50
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3,2001806458,0.700,121003,143001,b,1.00
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4,2001807012,0.240,121003,515591,d,1.25


In [21]:
# weight slab calculate by company x

def slab_calculater_X(d):
  if d['Zone_by_company'] == 'd':
    slab = np.ceil(d['totl_weight (Kg)']/1.25)*1.25
  elif d['Zone_by_company'] == 'a':
    slab = np.ceil(d['totl_weight (Kg)']/.25)*.25
  elif d['Zone_by_company'] == 'b':
    slab = np.ceil(d['totl_weight (Kg)']/.50)*.50
  elif d['Zone_by_company'] == 'c':
    slab = np.ceil(d['totl_weight (Kg)']/.75)*.75
  elif d['Zone_by_company'] == 'e':
    slab = np.ceil(d['totl_weight (Kg)']/1.00)*1.00
  
  return slab

final_df['weight_slab_by_X'] = final_df.apply(lambda x: slab_calculater_X(x), axis=1)

In [22]:
final_df

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),ExternOrderNo,totl_weight (Kg),Warehouse Pincode,Customer Pincode,Zone_by_company,weight_slab,weight_slab_by_X
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,2001806232,1.302,121003,507101,d,2.50,2.50
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2,2001806273,0.615,121003,486886,d,1.25,1.25
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,2001806408,2.265,121003,532484,d,2.50,2.50
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3,2001806458,0.700,121003,143001,b,1.00,1.00
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4,2001807012,0.240,121003,515591,d,1.25,1.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,1091118551656,2001812941,0.73,121003,325207,d,Forward charges,90.2,2001812941,0.500,121003,325207,b,1.25,0.50
120,1091117614452,2001809383,0.50,121003,303702,d,Forward and RTO charges,86.7,2001809383,0.607,121003,303702,b,1.25,1.00
121,1091120922803,2001820978,0.50,121003,313301,d,Forward charges,45.4,2001820978,0.515,121003,313301,b,1.25,1.00
122,1091121844806,2001811475,0.50,121003,173212,b,Forward charges,33.0,2001811475,0.689,121003,173212,e,0.50,1.00


In [23]:
# Calculate (Forword charges) and (Forword and RTO charges)

charge = []
for i in final_df.index:
  d =  final_df.iloc[i]
  if d['Zone_by_company'] =='d':
    if d['Type of Shipment'] == 'Forward charges':
        fixed_charge = 45.4
        additional_charge = 44.8*(d['weight_slab_by_X']/1.25-1)
        totel_charge = fixed_charge+additional_charge
        charge.append(totel_charge)
    elif d['Type of Shipment'] == 'Forward and RTO charges':
        fixed_charge = 45.4+41.3
        additional_charge = 2*44.8*(d['weight_slab_by_X']/1.25-1)
        totel_charge= fixed_charge+additional_charge
        charge.append(totel_charge)
  elif d['Zone_by_company'] =='b':
    if d['Type of Shipment'] == 'Forward charges':
        fixed_charge = 33
        additional_charge = 28.3*(d['weight_slab_by_X']/0.5-1)
        totel_charge = fixed_charge+additional_charge
        charge.append(totel_charge)
    elif d['Type of Shipment'] == 'Forward and RTO charges':
        fixed_charge = 33+20.5
        additional_charge = 2*28.3*(d['weight_slab_by_X']/0.5-1)
        totel_charge= fixed_charge+additional_charge
        charge.append(totel_charge)

  elif d['Zone_by_company'] =='e':
    if d['Type of Shipment'] == 'Forward charges':
        fixed_charge = 56.6
        additional_charge = 55.5*(d['weight_slab_by_X']-1)
        totel_charge = fixed_charge+additional_charge
        charge.append(totel_charge)
    elif d['Type of Shipment'] == 'Forward and RTO charges':
        fixed_charge = 56.6+50.5
        additional_charge = 2*55.5*(d['weight_slab_by_X']-1)
        totel_charge= fixed_charge+additional_charge
        charge.append(totel_charge)



In [24]:
final_df

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),ExternOrderNo,totl_weight (Kg),Warehouse Pincode,Customer Pincode,Zone_by_company,weight_slab,weight_slab_by_X
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,2001806232,1.302,121003,507101,d,2.50,2.50
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2,2001806273,0.615,121003,486886,d,1.25,1.25
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,2001806408,2.265,121003,532484,d,2.50,2.50
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3,2001806458,0.700,121003,143001,b,1.00,1.00
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4,2001807012,0.240,121003,515591,d,1.25,1.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,1091118551656,2001812941,0.73,121003,325207,d,Forward charges,90.2,2001812941,0.500,121003,325207,b,1.25,0.50
120,1091117614452,2001809383,0.50,121003,303702,d,Forward and RTO charges,86.7,2001809383,0.607,121003,303702,b,1.25,1.00
121,1091120922803,2001820978,0.50,121003,313301,d,Forward charges,45.4,2001820978,0.515,121003,313301,b,1.25,1.00
122,1091121844806,2001811475,0.50,121003,173212,b,Forward charges,33.0,2001811475,0.689,121003,173212,e,0.50,1.00


In [25]:
final_df.head()

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),ExternOrderNo,totl_weight (Kg),Warehouse Pincode,Customer Pincode,Zone_by_company,weight_slab,weight_slab_by_X
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,2001806232,1.302,121003,507101,d,2.50,2.50
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2,2001806273,0.615,121003,486886,d,1.25,1.25
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,2001806408,2.265,121003,532484,d,2.50,2.50
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3,2001806458,0.700,121003,143001,b,1.00,1.00
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4,2001807012,0.240,121003,515591,d,1.25,1.25


In [26]:
charge_df  = pd.DataFrame(charge,columns = ['charges'])
final_df = pd.concat([final_df,round(charge_df,2)],axis = 1)

**Calculate totel charge**

In [27]:
final_df['totel_charges'] = final_df['charges']

In [28]:
final_df['difference b/w charges'] =round((final_df['totel_charges'] -final_df['Billing Amount (Rs.)']) ,2)

In [29]:
final_df.sample(50)

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),ExternOrderNo,totl_weight (Kg),Warehouse Pincode,Customer Pincode,Zone_by_company,weight_slab,weight_slab_by_X,charges,totel_charges,difference b/w charges
92,1091117228133,2001806823,0.59,121003,314001,d,Forward charges,90.2,2001806823,0.127,121003,314001,b,1.25,0.50,33.0,33.0,-57.2
23,1091118547832,2001812838,0.60,121003,262405,b,Forward and RTO charges,102.3,2001812838,0.558,121003,262405,b,1.00,1.00,110.1,110.1,7.8
69,1091118009786,2001811809,0.50,121003,311011,d,Forward and RTO charges,86.7,2001811809,0.500,121003,311011,b,1.25,0.50,53.5,53.5,-33.2
82,1091121034641,2001821766,0.20,121003,313001,d,Forward charges,45.4,2001821766,0.240,121003,313001,b,1.25,0.50,33.0,33.0,-12.4
60,1091117324011,2001807241,0.79,121003,341001,d,Forward charges,90.2,2001807241,0.607,121003,341001,b,1.25,1.00,61.3,61.3,-28.9
6,1091117229776,2001806885,1.00,121003,208019,b,Forward charges,61.3,2001806885,0.840,121003,208019,b,1.00,1.00,61.3,61.3,0.0
54,1091117324394,2001807362,2.00,121003,452018,d,Forward charges,179.8,2001807362,2.016,121003,452018,d,2.50,2.50,90.2,90.2,-89.6
103,1091117437035,2001808679,0.72,121003,303903,d,Forward charges,90.2,2001808679,0.500,121003,303903,b,1.25,0.50,33.0,33.0,-57.2
26,1091120014461,2001818390,0.80,121003,783301,e,Forward and RTO charges,213.5,2001818390,0.841,121003,783301,e,1.00,1.00,107.1,107.1,-106.4
116,1091117905022,2001811058,0.72,121003,302018,d,Forward charges,90.2,2001811058,0.500,121003,302018,b,1.25,0.50,33.0,33.0,-57.2


##**Exctract desire column from final_df**

In [30]:
output_data_1_df = final_df[['Order ID','AWB Code','totl_weight (Kg)','weight_slab_by_X','Charged Weight','weight_slab','Zone_by_company','Zone','totel_charges','Billing Amount (Rs.)','difference b/w charges']]

In [31]:
output_data_1_df=output_data_1_df.rename(columns = {'totl_weight (Kg)e':'totel_weight (Kg)',
                                                    'totel_charges':' Expected Charge as per X (Rs.)',
                                                    'Zone':'zone_courier_company',
                                                    'totel_charges':'Expected Charges'})

##**Find the count and sum of correctly_charge,overcharge,undercharge**

In [32]:
correctly_charge = []
overcharged = []
undercharged =  []
for i in final_df.index:
  d = final_df.iloc[i]
  if d['difference b/w charges'] == 0:
    correctly_charge.append(d['totel_charges'])
  elif d['difference b/w charges'] <0:
    overcharged.append(d['difference b/w charges'])
  elif d['difference b/w charges'] >0:
    undercharged.append(d['difference b/w charges'])

In [33]:
output_data_1_df

,Order ID,AWB Code,totl_weight (Kg),weight_slab_by_X,Charged Weight,weight_slab,Zone_by_company,zone_courier_company,Expected Charges,Billing Amount (Rs.),difference b/w charges
0,2001806232,1091117222124,1.302,2.50,1.30,2.50,d,d,90.2,135.0,-44.8
1,2001806273,1091117222194,0.615,1.25,1.00,1.25,d,d,45.4,90.2,-44.8
2,2001806408,1091117222931,2.265,2.50,2.50,2.50,d,d,90.2,224.6,-134.4
3,2001806458,1091117223244,0.700,1.00,1.00,1.00,b,b,61.3,61.3,0.0
4,2001807012,1091117229345,0.240,1.25,0.15,1.25,d,d,45.4,45.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...
119,2001812941,1091118551656,0.500,0.50,0.73,1.25,b,d,33.0,90.2,-57.2
120,2001809383,1091117614452,0.607,1.00,0.50,1.25,b,d,110.1,86.7,23.4
121,2001820978,1091120922803,0.515,1.00,0.50,1.25,b,d,61.3,45.4,15.9
122,2001811475,1091121844806,0.689,1.00,0.50,0.50,e,b,56.6,33.0,23.6


##**Export dataset in  to Excel sheet**

In [34]:
output_data_1_df.to_excel('output_1.xlsx')

In [35]:
output_data_1_df.head(30)

,Order ID,AWB Code,totl_weight (Kg),weight_slab_by_X,Charged Weight,weight_slab,Zone_by_company,zone_courier_company,Expected Charges,Billing Amount (Rs.),difference b/w charges
0,2001806232,1091117222124,1.302,2.50,1.30,2.50,d,d,90.2,135.0,-44.8
1,2001806273,1091117222194,0.615,1.25,1.00,1.25,d,d,45.4,90.2,-44.8
2,2001806408,1091117222931,2.265,2.50,2.50,2.50,d,d,90.2,224.6,-134.4
3,2001806458,1091117223244,0.700,1.00,1.00,1.00,b,b,61.3,61.3,0.0
4,2001807012,1091117229345,0.240,1.25,0.15,1.25,d,d,45.4,45.4,0.0
5,2001806686,1091117229555,0.240,1.25,0.15,1.25,d,d,45.4,45.4,0.0
6,2001806885,1091117229776,0.840,1.00,1.00,1.00,b,b,61.3,61.3,0.0
7,2001807058,1091117323112,1.168,1.50,1.15,1.50,b,b,89.6,89.6,0.0
8,2001807186,1091117323812,0.500,1.25,0.50,1.25,d,d,45.4,45.4,0.0
9,2001807290,1091117324206,0.500,1.25,0.50,1.25,d,d,45.4,45.4,0.0


#**Final Result**

In [36]:
print(f"Total orders where X has been correctly charged  is {len(correctly_charge)} and correctly charged amount is {round(sum(correctly_charge),2)}")

Total orders where X has been correctly charged  is 13 and correctly charged amount is 774.0


In [37]:
print(f'Total Orders where X has been overcharged is  {len(overcharged)} and overcharged amount is {round(sum(overcharged),2)}')

Total Orders where X has been overcharged is  101 and overcharged amount is -6377.5


In [38]:
print(f'Total Orders where X has been undercharged is {len(undercharged)}  and undercharged amount is {round(sum(undercharged),2)}')

Total Orders where X has been undercharged is 10  and undercharged amount is 232.1
